LAYOUT :)   

                 EDA       :
                 
                 PIPELINE  :
                 
                              ML MODELS :)
                              LinearRegression
                                                          
                              XGBoostingRegressor
                              XGBoostingRegressor with optuna
                              
                 ALL IN PLOT:
                              RMSE of All MOdels
                 PREDICTIONS:

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Datasets Loading....

In [ ]:
train_df = pd.read_csv('/kaggle/input/tabular-playground-series-feb-2021/train.csv')
test_df = pd.read_csv('/kaggle/input/tabular-playground-series-feb-2021/test.csv')


In [ ]:
#train_df2  = train_df[train_df.target<9.523992259999998]
#train_df2

In [ ]:
print(train_df.shape,test_df.shape)

In [ ]:
df = pd.concat([train_df.drop(['target','id'],axis=1),test_df.drop(['id'],axis=1)],axis=0)
df.head()

## EDA

In [ ]:
df.isnull().sum()

In [ ]:
def divideFeatures(df):
    numerical_features = df.select_dtypes(include=[np.number])
    categorical_features = df.select_dtypes(include=[np.object])
    return numerical_features, categorical_features

In [ ]:
cont_features, cat_features =divideFeatures(df)

In [ ]:
# boxplots of numerical features for outlier detection

fig = plt.figure(figsize=(16,16))
for i in range(len(cont_features.columns)):
    fig.add_subplot(5, 4, i+1)
    sns.boxplot(y=cont_features.iloc[:,i])
plt.tight_layout()
plt.show()

In [ ]:
# distplots for categorical data

fig = plt.figure(figsize=(16,10))
for i in range(len(cat_features.columns[:])):
    fig.add_subplot(5, 4, i+1)
    cat_features.iloc[:,i].hist()
    plt.xlabel(cat_features.columns[i])
plt.tight_layout()
plt.show()

In [ ]:
# correlation heatmap for all features
plt.figure(figsize=(18,10),dpi=200)
corr = train_df.corr()
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
sns.heatmap(corr, mask = mask, annot=True)

plt.show()

In [ ]:
# plot sample skewed feature
plt.figure(figsize=(10,4))
sns.distplot(train_df['target'])
plt.show()

In [ ]:
skewed_features = cont_features.apply(lambda x: x.skew()).sort_values(ascending=False)
skewed_features

import warnings
warnings.filterwarnings("ignore")
fig = plt.figure(figsize=(16,30))
for i in range(len(cont_features.columns)):
    fig.add_subplot(10,5,i+1)
    sns.distplot(cont_features.iloc[:,i])
    plt.xlabel(cont_features.columns[i])
plt.tight_layout()
plt.show();
    

In [ ]:
print(train_df.shape,test_df.shape,df.shape)

## Modeling

In [ ]:
from sklearn.preprocessing import OneHotEncoder , LabelEncoder
from sklearn.pipeline import make_pipeline ,Pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.compose import make_column_transformer ,ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split ,cross_val_score,validation_curve,learning_curve
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV ,RandomizedSearchCV
from sklearn.tree import plot_tree
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor,GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LogisticRegression


In [ ]:
one = OneHotEncoder(handle_unknown='ignore')
scaler =StandardScaler()

fs = SelectKBest()
le =LabelEncoder()

In [ ]:
print(df.select_dtypes(include='object').columns)
df.select_dtypes(exclude='object').columns

In [ ]:
cat_features.columns


In [ ]:
def lencode(df):
    for i in df.columns:
        for k in cat_features.columns:
            if i ==k:
                df[i]= le.fit_transform(df[i])
        
        
        

In [ ]:
lencode(df)
df.head()


In [ ]:
X_train = df[:300000] 
Xf_test = df[300000:]
y = train_df['target']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y, test_size=0.1, random_state=42)

In [ ]:
#ct = make_column_transformer((one,['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8','cat9']),
                             #(scaler,['id', 'cont0', 'cont1', 'cont2', 'cont3', 'cont4', 'cont5', 'cont6','cont7', 'cont8', 'cont9', 'cont10', 'cont11', 'cont12', 'cont13']),
                              #remainder='passthrough')

In [ ]:
X_train.head()

In [ ]:

linear = LinearRegression()
random = RandomForestRegressor(n_estimators=100,random_state=42,n_jobs=-1,max_features=4)


In [ ]:
import xgboost as xgb
xgbr = xgb.XGBRegressor(tree_method='gpu_hist')


In [ ]:
import optuna
def objective(trial):
    params = {
        'random_state': 0,
        'n_estimators': trial.suggest_categorical('n_estimators', [10000]),
        'max_depth': trial.suggest_int('max_depth', 3, 8),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 10),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 10),
        'gamma': trial.suggest_float('gamma', 0.0, 10),
        'subsample': trial.suggest_categorical('subsample', [0.8, 0.9, 1.0]),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.1, 0.2, 0.3, 0.4, 0.5]),
        'tree_method':'gpu_hist'    # comment this line if GPU is off
    }
    model = xgb.XGBRegressor(**params) 
    model.fit(X_train, y_train, eval_set=[(X_test,y_test)], early_stopping_rounds=1000, verbose=0)
    y_pred = model.predict(X_test)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    
    return rmse

In [ ]:
%%time
study = optuna.create_study(direction='minimize',sampler=optuna.samplers.TPESampler(seed=0))
study.optimize(objective, n_trials=100)
print('Number of finished trials:', len(study.trials))
print('Best parameters:', study.best_trial.params)
print('Best RMSE:', study.best_trial.value)

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
params = study.best_params
params['random_state'] = 0
params['n_estimators'] = 10000
params['tree_method'] = 'gpu_hist'

model3 = xgb.XGBRegressor(**params)
model3.fit(X_train,y_train,eval_set=[(X_test, y_test)],early_stopping_rounds=1000,verbose=2)


In [ ]:

model = make_pipeline(linear)
model.fit(X_train,y_train)

model2 = make_pipeline(xgbr)
model2.fit(X_train,y_train)

In [ ]:
models ={'linear':model}
models['xgbr'] = model2
models['optuna_xbg'] =model3

In [ ]:
def rmse(y_test,y_pred):
    rmse= np.sqrt(mean_squared_error(y_test,y_pred))
    return rmse


In [ ]:
from sklearn.model_selection import KFold
from sklearn.base import clone

def cross_val_rmse(model):
    model =clone(model)
    five_fold = KFold(n_splits=5)
    rmse_val =[]
    for tr_ind,val_ind in five_fold.split(X_train):
        model.fit(X_train.iloc[tr_ind,:],y.iloc[tr_ind])
        rmse_val.append(rmse(y.iloc[val_ind],model.predict(X_train.iloc[val_ind,:])))
        
    return np.mean(rmse_val)

In [ ]:
import plotly.graph_objects as go

def compare_models(models):
    training_rmse = [rmse(y_train,model.predict(X_train)) for model in models.values()]
    validate_rmse = [cross_val_rmse(model) for model in models.values()] 
    testing_rmse = [rmse(y_test,model.predict(X_test)) for model in models.values()]
    names = list(models.keys())
    fig = go.Figure([
                      go.Bar(x=names ,y=training_rmse ,name='traing_rmse'),
                      go.Bar(x=names ,y=validate_rmse ,name='validate_rmse'),
                      go.Bar(x=names ,y=testing_rmse ,name='testing_rmse',opacity=.3)])
    return fig

In [ ]:
fig = compare_models(models)
fig.update_yaxes(range=[0,2],title="RMSE")
fig.show()

In [ ]:
y_pred = model3.predict(X_test)
y_pred =np.round(y_pred,6)
print('min',y_pred.min())
print('max',y_pred.max())
RMSE = np.sqrt(mean_squared_error(y_test,y_pred))
print('RMSE',RMSE)
print(y_pred.tolist()) 

## submit

In [ ]:
ys_pred =model3.predict(Xf_test)
ys_pred =np.round(ys_pred,6)
ys_pred.tolist()

In [ ]:
sub = pd.DataFrame({'id':test_df.id,'target':ys_pred})
sub.to_csv('TPS.csv',index=False)
sub.head()

## Upvote if u like :)